In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import ITunesAmazonStructuredBenchmark

benchmark = ITunesAmazonStructuredBenchmark(data_dir_path="../data/")
benchmark

17:39:24 INFO:Extracting iTunes-Amazon-Structured...
17:39:24 INFO:Reading iTunes-Amazon-Structured row_dict...
17:39:24 INFO:Reading iTunes-Amazon-Structured train.csv...
17:39:24 INFO:Reading iTunes-Amazon-Structured valid.csv...
17:39:24 INFO:Reading iTunes-Amazon-Structured test.csv...


<ITunesAmazonStructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/iTunes-Amazon/itunes_amazon_exp_data.zip

## Preprocess

In [5]:
field_list = ['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released']

In [6]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for record_dict in [benchmark.train_record_dict, benchmark.valid_record_dict, benchmark.test_record_dict]:
    for record in record_dict.values():
        for field in field_list:
            record[field] = clean_str(record[field])

  0%|          | 0/62830 [00:00<?, ?it/s]

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [10]:
from entity_embed import PairNumericalizer

pair_numericalizer = PairNumericalizer(field_list)

17:39:25 INFO:For attr=Song_Name, computing actual max_str_len
17:39:25 INFO:actual_max_str_len=21 must be even to enable NN pooling. Updating to 22
17:39:25 INFO:For attr=Song_Name, using actual_max_str_len=22
17:39:25 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:39:29 INFO:For attr=Artist_Name, computing actual max_str_len
17:39:29 INFO:For attr=Artist_Name, using actual_max_str_len=20
17:39:29 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:39:33 INFO:For attr=Album_Name, computing actual max_str_len
17:39:33 INFO:For attr=Album_Name, using actual_max_str_len=16
17:39:33 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:39:37 INFO:For attr=Genre, computing actual max_str_len
17:39:37 INFO:actual_max_str_len=13 must be even to enable NN pooling. Updating to 14
17:39:37 INFO:For attr=Genre, using actual_max_str_len=14
17:39:37 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:39:40 INFO:For attr=Price, computing actual max_str_len
17:39:40 INFO:F

{'Song_Name': AttrConfig(source_attr='Song_Name', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=22, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_Song_Name': AttrConfig(source_attr='Song_Name', field_type=<FieldType.SEMANTIC: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!

In [11]:
batch_size = 32
eval_batch_size = 256
datamodule = benchmark.build_matcher_datamodule(
    pair_numericalizer=pair_numericalizer,
    batch_size=batch_size,
    eval_batch_size=eval_batch_size,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import Matcher

model = Matcher(
    pair_numericalizer=pair_numericalizer
)

In [13]:
trainer = model.fit(
    datamodule,
    min_epochs=5,
    max_epochs=100,
    check_val_every_n_epoch=1,
    early_stop_monitor="valid_f1_at_0.5",
    tb_save_dir='../tb_logs',
    tb_name=f'matcher-{benchmark.dataset_name}'
)

GPU available: True, used: True
17:39:41 INFO:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
17:39:41 INFO:TPU available: None, using: 0 TPU cores


In [ ]:
model.validate(datamodule)

## Testing

In [17]:
model.test(datamodule)

17:42:45 INFO:Train pair count: 78
17:42:45 INFO:Valid pair count: 27
17:42:45 INFO:Test pair count: 27
17:42:45 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{'test_f1_at_0.3': 0.7941176470588235,
  'test_f1_at_0.5': 0.9310344827586207,
  'test_f1_at_0.7': 0.9473684210526316,
  'test_f1_at_0.9': 0.9259259259259259,
  'test_pair_entity_ratio_at_0.3': 0.7592592592592593,
  'test_pair_entity_ratio_at_0.5': 0.5740740740740741,
  'test_pair_entity_ratio_at_0.7': 0.5555555555555556,
  'test_pair_entity_ratio_at_0.9': 0.5,
  'test_precision_at_0.3': 0.6585365853658537,
  'test_precision_at_0.5': 0.8709677419354839,
  'test_precision_at_0.7': 0.9,
  'test_precision_at_0.9': 0.9259259259259259,
  'test_recall_at_0.3': 1.0,
  'test_recall_at_0.5': 1.0,
  'test_recall_at_0.7': 1.0,
  'test_recall_at_0.9': 0.9259259259259259}]